In [ ]:
# TODO: load pkl, combine to one big dataset, train-val-test split (stratify by image), try hest or conch if time allows, finish report (merge with previous)
# TODO: optional, check colab tutorial works